In [2]:
import polars as pl
import pyodbc

In [4]:
wd = "E:/Users/jhernandezr/OneDrive - Nacional Financiera, S.N.C/Archivos de Itzel Romo Cruz - Garantías/Producto Nafin/Análisis spread/"
wd_foto = "E:/Users/jhernandezr/OneDrive - Nacional Financiera, S.N.C/Archivos de Itzel Romo Cruz - Garantías/Reporte/2024/2024_06/"

fl_spread = wd + "Análisis_Spread_202506.xlsx"
fl_spread_ant = wd + "Análisis_Spread_202406.xlsx"
fl_spread_ant_access = wd_foto + "FOTOS_R-NR_Jun24.accdb"

### Importar y exportar access to csv

In [ ]:
sql = """
SELECT 
 FOTOS.FECHA_VALOR, 
 FOTOS.BANCO, 
 FOTOS.BUCKET, 
 FOTOS.Tasa_Id, 
 FOTOS.Tasa_Interes, 
 FOTOS.[SALDO (MDP)] AS SALDO_MDP, 
 FOTOS.[MONTO CREDITO (MDP)] AS MONTO_CREDITO_MDP, 
 FOTOS.[MGI (MDP)] AS MGI_MDP, 
 FOTOS.[MPAGADO (MDP)] AS M_PAGADO_MDP, 
 FOTOS.[RECUPERADOS (MDP)] AS M_RECUPERADO_MDP, 
 FOTOS.[RESCATADOS (MDP)] AS M_RESCATADO_MDP, 
 FOTOS.PLAZO,
 FOTOS.PORCENTAJE_COMISION_GARANTIA,
 FOTOS.ESTADO_ID,
 FOTOS.DESCRIBE_DESREC,
 FOTOS.ESTRATO_ID,
 FOTOS.SECTOR_ID
FROM 
(SELECT * 
FROM Foto_VF
WHERE AGRUPAMIENTO_ID = 10000)  AS FOTOS ;
"""

def read_access(file, sql):
    conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + file
    conn = pyodbc.connect(conn_str)
    df = pl.read_database(query=sql, connection=conn)
    conn.close()
    return df

df2 = read_access(fl_spread_ant_access, sql)
df2.write_csv("pn2.csv")

In [5]:
df = pl.read_excel(fl_spread, sheet_name="Datos")
df = df[:,:-2]

Could not determine dtype for column 23, falling back to string
Could not determine dtype for column 24, falling back to string


In [6]:
df3 = pl.read_excel(fl_spread_ant, sheet_name="Datos")
df3 = df3[:,:-2]

Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 23, falling back to string
Could not determine dtype for column 24, falling back to string


In [7]:
# Correlacion General
(df.filter(pl.col("SALDO_MDP")==0)
 .group_by("BUCKET")
 .agg(
     pl.col("SPREAD").mean().alias("spread"),
     pl.col("PO").mean().alias("PO"))
 #.select(["spread", "PO"]).corr()
 #.select("PO")[0].item()
 )

BUCKET,spread,PO
i64,f64,f64
1,0.04435,0.039237
3,0.088004,0.054659
5,0.186573,0.065553
4,0.121495,0.068638
2,0.064204,0.040208


In [63]:
(df3.filter(pl.col("SALDO_MDP")==0)
 .group_by("BUCKET")
 .agg(
     pl.col("SPREAD").mean().alias("spread"),
     pl.col("PO").mean().alias("PO"))
 .select(["spread", "PO"]).corr()
 .select("PO")[0].item()
 )

0.8999494713839442

In [26]:
def calcula_correlacion(df, banco=""):

    if banco != "":
        df = df.filter(pl.col("BANCO")==banco)#.filter(pl.col("SALDO_MDP")==0)


    corr = (df
    .group_by("BUCKET")
    .agg(
        pl.col("SPREAD").mean().alias("spread"),
        pl.col("PO").mean().alias("PO"))
    .select(["spread", "PO"]).corr()
    #.select("PO")[0].item()
    )
    return corr

calcula_correlacion(df, "BBV BANCOMER")

spread,PO
f64,f64
1.0,-0.359512
-0.359512,1.0


In [29]:
def spread_por_bucket(df, banco=""):


    corr = (df
    .group_by(["BANCO","BUCKET"])
    .agg(
        pl.col("SPREAD").mean().alias("spread"),
        pl.col("PO").mean().alias("PO"))
    #.select(["BUCKET", "PO"]).corr()
    #.select("PO")[0].item()
    ).sort(["BANCO", "BUCKET"])
    return corr

spread_po = spread_por_bucket(df3)

In [30]:
spread_po.write_csv("spread_po_2024.csv")

In [21]:
(df
 .group_by("BANCO")
 .agg(
     pl.col("SALDO_MDP").sum(),
     pl.col("MGI_MDP").sum(),
     pl.col("M_PAGADO_MDP").sum(),
     pl.col("PO").mean(),
     pl.col("SPREAD").mean()
 )
 ).write_csv("saldos.csv")

In [59]:
bancos = [
    #'AF BANREGIO (SOFOM)',
    'BANAMEX',
    'BANCA AFIRME',
    'BANCA MIFEL',
    #'BANCO AHORRO FAMSA SA INSTITUCION BANCARIA',
    'BANCO DEL BAJIO',
    'BANCO INVERLAT MEXICO BROKER',
    'BANCO MERCANTIL DEL NORTE',
    'BANCO MULTIVA SA DE CV',
    'BANCO REGIONAL DE MONTERREY',
    'BANCO SANTANDER',
    'BANCO VE POR MAS',
    'BANCOPPEL SA INSTITUCION DE BANCA MULTIPLE',
    'BANSI SA',
    'BBV BANCOMER',
    'FINANCIERA BAJIO SA DE CV',
    'HSBC MEXICO'
]
for banco in bancos:
    print(banco, ": ", calcula_correlacion(df, banco), ";")

BANAMEX :  -0.5301250406879578 ;
BANCA AFIRME :  0.7755522554406108 ;
BANCA MIFEL :  -0.3881741226270517 ;
BANCO DEL BAJIO :  0.7701219799125647 ;
BANCO INVERLAT MEXICO BROKER :  -0.27035732849682964 ;
BANCO MERCANTIL DEL NORTE :  -0.13121905646196513 ;
BANCO MULTIVA SA DE CV :  0.9946066711686606 ;
BANCO REGIONAL DE MONTERREY :  0.67654077863017 ;
BANCO SANTANDER :  0.9858192864243968 ;
BANCO VE POR MAS :  -0.9999999999999999 ;
BANCOPPEL SA INSTITUCION DE BANCA MULTIPLE :  -0.8468336186604299 ;
BANSI SA :  -1.0 ;
BBV BANCOMER :  -0.4660941613304094 ;
FINANCIERA BAJIO SA DE CV :  0.9180639892661828 ;
HSBC MEXICO :  0.9018688085046442 ;


In [60]:
for banco in bancos:
    print(banco, ": ", calcula_correlacion(df3, banco), ";")

BANAMEX :  -0.6063483460396376 ;
BANCA AFIRME :  0.8595359389873057 ;
BANCA MIFEL :  0.11114951843078982 ;
BANCO DEL BAJIO :  0.7655036537656684 ;
BANCO INVERLAT MEXICO BROKER :  0.01824020250422989 ;
BANCO MERCANTIL DEL NORTE :  -0.31304288865961705 ;
BANCO MULTIVA SA DE CV :  0.9969644728445411 ;
BANCO REGIONAL DE MONTERREY :  0.628744994701132 ;
BANCO SANTANDER :  0.9795166653631994 ;
BANCO VE POR MAS :  -1.0 ;
BANCOPPEL SA INSTITUCION DE BANCA MULTIPLE :  -0.8176303485108952 ;
BANSI SA :  -1.0 ;
BBV BANCOMER :  0.36914563769513214 ;
FINANCIERA BAJIO SA DE CV :  0.7050306169980757 ;
HSBC MEXICO :  0.8444615125906381 ;


In [39]:
(df3
.group_by("BUCKET")
.agg(
    #pl.col("SPREAD").mean().alias("spread"),
    pl.col("PO").mean().alias("PO"))
#.select(["spread", "PO"]).corr()
#.select("PO")[0].item()
).sort("BUCKET")

BUCKET,PO
i64,f64
1,0.021118
2,0.023644
3,0.037071
4,0.047197
5,0.041837


In [40]:
(df
.group_by("BUCKET")
.agg(
    #pl.col("SPREAD").mean().alias("spread"),
    pl.col("PO").mean().alias("PO"))
#.select(["spread", "PO"]).corr()
#.select("PO")[0].item()
).sort("BUCKET")

BUCKET,PO
i64,f64
1,0.025174
2,0.025145
3,0.037843
4,0.046923
5,0.040189
